<a href="https://colab.research.google.com/github/Thing-ji/data_analysis_reference/blob/main/folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

import seaborn as sns
import matplotlib.pyplot as plt

import folium
from folium import plugins
# from folium.plugins import HeatMap

import json # json.org : 효율적으로 데이터를 저장하고 교환(exchange data)하는데 사용하는 텍스트 데이터 포맷
from datetime import datetime 

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
patient = pd.read_csv('PatientInfo.csv')
region = pd.read_csv('Region.csv')

FileNotFoundError: ignored

In [ ]:
patient

In [ ]:
#patient.head()

In [ ]:
#egion.head()

# 1. plotting just a map (제일 먼저, 그냥 지도 띄우기)


In [ ]:
center = [37, 128]
folium.Map(location = center, zoom_start = 7, tiles = 'Stamen Terrain')
#folium.Map(location = center, zoom_start = 7, tiles = 'Stamen Toner')

There are several types of map, default, Stamen Terrain, and Stamen Toner.

In [ ]:
m = folium.Map([36, 128], zoom_start = 7)
m # this map is default style

Additional tool : Fullscreen, MousePosition(get coodinates)

추가 기능: 풀스크린으로 만들기, 마우스 갖다대면 좌표드게 하기.


In [ ]:
center = [36, 128]
m = folium.Map(location = center, zoom_start = 7)

plugins.Fullscreen(position = 'topright',  # Full screen
                   title = 'Click to Expand',
                   title_cancel = 'Click to Exit',
                   force_separate_button = True).add_to(m)

plugins.MousePosition().add_to(m) ## you can easily get coordinates (쉽게 좌표를 얻을 수 있음)

In [ ]:
m

# 2. MarkerCluster : Lets see the number of regional confirmed patient.

지역별로 몇명의 확진 환자가 나왔는지 한번 확인해보자.

In [ ]:
patient.head()

,patient_id,sex,age,country,province,city,infection_case,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,male,50s,Korea,Seoul,Gangseo-gu,overseas inflow,NaN,75,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,male,30s,Korea,Seoul,Jungnang-gu,overseas inflow,NaN,31,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,male,50s,Korea,Seoul,Jongno-gu,contact with patient,2002000001,17,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,male,20s,Korea,Seoul,Mapo-gu,overseas inflow,NaN,9,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,female,20s,Korea,Seoul,Seongbuk-gu,contact with patient,1000000002,2,NaN,2020-01-31,2020-02-24,NaN,released


In [ ]:
region.head()

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739
1,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088
2,10020,Seoul,Gangdong-gu,37.530492,127.123837,27,32,0,1.54,14.55,5.4,1023
3,10030,Seoul,Gangbuk-gu,37.639938,127.025508,14,21,0,0.67,19.49,8.5,628
4,10040,Seoul,Gangseo-gu,37.551166,126.849506,36,56,1,1.17,14.39,5.7,1080


In [ ]:
regional_patient = pd.merge(patient[['patient_id','confirmed_date', 'sex', 'age', 'province', 'city']],
                            region[['province', 'city', 'latitude', 'longitude']], how = 'left', on = ['province', 'city'])
display(regional_patient.head())
print('There are a few missing values')
print('약간의 missing 값들이 있다. 분석할 때 유의해야할 것 같다.')
display(regional_patient.isnull().sum()/regional_patient.shape[0])

,patient_id,confirmed_date,sex,age,province,city,latitude,longitude
0,1000000001,2020-01-23,male,50s,Seoul,Gangseo-gu,37.551166,126.849506
1,1000000002,2020-01-30,male,30s,Seoul,Jungnang-gu,37.606832,127.092656
2,1000000003,2020-01-30,male,50s,Seoul,Jongno-gu,37.572999,126.979189
3,1000000004,2020-01-30,male,20s,Seoul,Mapo-gu,37.566283,126.901644
4,1000000005,2020-01-31,female,20s,Seoul,Seongbuk-gu,37.589562,127.016700


There are a few missing values
약간의 missing 값들이 있다. 분석할 때 유의해야할 것 같다.


patient_id        0.000000
confirmed_date    0.000581
sex               0.217231
age               0.267183
province          0.000000
city              0.018199
latitude          0.042401
longitude         0.042401
dtype: float64

In [ ]:
import sys
sys.setrecursionlimit(1500)

In [ ]:
regional_count = regional_patient[['latitude', 'longitude']].dropna() # drop missing values

m = folium.Map(location = center, zoom_start = 7)

plugins.MousePosition().add_to(m)
plugins.MarkerCluster(regional_count).add_to(m) # 재귀한계 문제걸림. 해결하기.

m

RecursionError: ignored

# 3. Make subgroups to plot : Female / Male
여성, 남성 감염자를 하위그룹으로 하여 따로다로 볼 수 있도록 지도를 그려본다.

In [ ]:
male_patient = regional_patient\
.query('sex in ("male")')[['latitude', 'longitude']].dropna()

female_patient = regional_patient\
.query('sex in ("female")')[['latitude', 'longitude']].dropna()

m = folium.Map(location = center, zoom_start = 7)

initial = folium.plugins.MarkerCluster(control = False)

m.add_child(initial)

female_g = plugins.FeatureGroupSubGroup(initial, 'Female')
m.add_child(female_g)

male_g = plugins.FeatureGroupSubGroup(initial, 'Male')
m.add_child(male_g)

plugins.MarkerCluster(female_patient).add_to(female_g)
plugins.MarkerCluster(male_patient).add_to(male_g)

folium.LayerControl(collapsed = False).add_to(m)

m

# 4. Make a Heat map : Instead of the number

지역별로 얼마나 많은 감염자가 있는지, 숫자로 보는 것이 아닌, heat map을 활용하여 시각적으로 나타내보겟다.

In [ ]:
folium.plugins.HeatMap(data, name=None, min_opacity=0.5, 
                       max_zoom=18, max_val=1.0, radius=25, blur=15, 
                       gradient=None, overlay=True, control=True, show=True, **kwargs)

- min_opacity (default 1.) – The minimum opacity the heat will start at.
- max_zoom (default 18) – Zoom level where the points reach maximum intensity (as intensity scales with zoom), equals maxZoom of the map by default
- max_val (float, default 1.) – Maximum point intensity
- radius (int, default 25) – Radius of each “point” of the heatmap
- blur (int, default 15) – Amount of blur
- gradient (dict, default None) – Color gradient config. e.g. {0.4: ‘blue’, 0.65: ‘lime’, 1: ‘red’}
- overlay (bool, default True) – Adds the layer as an optional overlay (True) or the base layer (False).
- control (bool, default True) – Whether the Layer will be included in LayerControls.
- show (bool, default True) – Whether the layer will be shown on opening (only for overlays).

##### Heatmap parameter

In [ ]:
heat_data = regional_patient\
.groupby(['latitude', 'longitude'])['patient_id'].count().reset_index().values.tolist()

m = folium.Map(location = center, zoom_start = 7) 

folium.plugins.HeatMap(heat_data).add_to(m)
m

In [ ]:
heat_data = region[['latitude', 'longitude', 'nursing_home_count']].values.tolist() # nursing home count

mark_data = regional_patient[['latitude', 'longitude']].dropna() # drop missing values

m = folium.Map(location = center, zoom_start = 7)

plugins.MarkerCluster(mark_data).add_to(m) # 통계량 표시

folium.plugins.HeatMap(heat_data, max_opacity = 0.3).add_to(m)

# 5. Heatmap with time : time series heat map

In [ ]:
folium.plugins.HeatMapWithTime(data, index = None, name = None,
                               radius = 15, min_opacity = 0,
                               max_opacity = 0.6, scale_radius=False, gradient=None, 
                               use_local_extrema=False, auto_play=False, 
                               display_index=True, index_steps=1, min_speed=0.1, 
                               max_speed=10, speed_step=0.1, position='bottomleft', 
                               overlay=True, control=True, show=True)
                               

Parameter what i find important

data – The points you want to plot. The outer list corresponds to the various time steps in sequential order. (weight is in (0, 1] range and defaults to 1 if not specified for a point)

index  – the same length as data, or is replaced by a simple count if not specified.

radius (default 15.) – The radius used around points for the heatmap.

min_opacity (default 0) – The minimum opacity for the heatmap.

max_opacity (default 0.6) – The maximum opacity for the heatmap.

scale_radius (default False) – Scale the radius of the points based on the zoom level.

auto_play (default False) – Automatically play the animation across time.


In [ ]:
#prepare data

regional_patient_bytime = regional_patient\
.groupby(['confirmed_date', 'latitude', 'longitude'])['patient_id'].count().reset_index()\
.sort_values(by = 'confirmed_date')

# data_range : 함수를 쓰면 날짜/시간을 나타내는 문자열을 자동으로 datetime 자료형으로 바꾼 후 DatetimeIndex 자료형 인덱스를 생성한다.
date_rng = pd.date_range(regional_patient_bytime['confirmed_date'].min(), regional_patient_bytime['confirmed_date'].max())
date_rng = [str(i)[:10] for i in date_rng]

# numpy.repeat 함수는 어레이의 요소들을 지정한 횟수만큼 반복한다.
# numpy.tile 함수는 어레이를 (타일과 같이) 지정한 횟수만큼 반복한다.
all_date_region = pd.DataFrame({'confirmed_date': np.repeat(date_rng, region[['latitude', 'longitude']].drop_duplicates().shape[0]),
                                'latitude': np.tile(list(region['latitude'].drop_duplicates()), len(date_rng)),
                                'longitude': np.tile(list(region['longitude'].drop_duplicaties()), len(date_rng))})

all_date_region = pd.merge(all_date_region, regional_patient_bytime, on = ['confirmed_date', 'latitude', 'longitude'], how = 'left').fillna(0)
all_date_region['cumsum'] = all_date_region.groupby(['latitude', 'longitude'])['patient_id'].cumsum() #  cumsum은 배열에서 주어진 축에 따라 누적되는 원소들의 누적 합을 계산하는 함수

all_date_region = all_date_region[all_date_region['cumsum'] != 0]

time_index, data = [], []
 # query를 [column] 대신 굳이 사용하는 이유는 방대한 데이터를 빠르게 처리할 수 있음이다.
for date in all_date_region['confirmed_date'].unique():
  time_index += [date[6:0].replace('-', '/')]
  time_list = all_date_region.query('confirmed_date == @date')[['latitude', 'longitude', 'cumsum']].values.tolist()
  data += [temp_list]
  del temp_list

HeatMapWithTime needs this type of input data

In [ ]:
data[:5], time_index[:5]

NameError: ignored

In [ ]:
# make a map

m = folium.Map([36, 128], zoom_start = 7) #, tiles = 'Mapbox Bright')

hm = folium.plugins.HeatMapWithTime(data, index = time_index, auto_play = False, min_opacity = 0.3, radius = 25, )

hm.add_to(m)

for bottom, click in zip([60, 90, 120, 150, 180], ['Loop', 'Forward', 'Play', 'Reverse', 'Backward']):

  # Thank you for html code. https://www.kaggle.com/poonsfc5/ds4c-covid-19-in-korea-eda-with-geo-data

   name = '''
            <div style="position: fixed; bottom: '''+str(bottom)+'''px; left: 50px; width: 100px; height: 29px; 
                        background-color: white; border:1px solid grey; z-index:9999; font-size:11px;text-align:center;"
                        >&nbsp; <br><b>Click : '''+ click +'''</b></div>'''  

    m.get_root().html.add_child(folium.Element(name))

m


# 6. Make Circles with timebar : Timestramped Geojson

In [ ]:
TimestrampedGeoJson(data, transition_time = 200, loop = True, auto_play = True,
                    add_last_point = True, period = 'P1D', min_speed = 0.1, max_speed = 10,
                    loop_button = False, date_options = 'YYYY-MM-DD HH:mm:ss',
                    time_slider_drag_update = Falsem duration = None)

parameter i find important

period (str, default "P1D") –  ‘P1M’ 1/month, ‘P1D’ 1/day, ‘PT1H’ 1/hour, and ‘PT1M’ 1/minute

In [ ]:
# prepare data
df_dropna = regional_patient.dropna()
df_dropna['timestamp'] = [i+'T00:00:00' for i in df_dropna['confirmed_date']]

points = []
for date in sorted(df_dropna['confirmed_date'].unique()):
  temp = df_dropna.query('confiremd_date == @date')

  temp_dict = {}
  temp_dict['coordinates'] = temp[['longitude', 'latitude']].values.tolist()

  points += [temp_dict]

features = [ { 'type': 'Feature', 
                  'geometry': { 'type': 'MultiPoint', 
                               'coordinates': point['coordinates'], }, 
                  'properties': { 'times': point['dates'],
                                 'icon' : 'circle'}
             } for point in points ] 

TimestampedGeoJson needs this type of input data.

In [ ]:
features[:3]

In [ ]:
m = folium.Map( location=[36, 128], zoom_start=7 )

plugins.TimestampedGeoJson({ 
    'type': 'FeatureCollection', 
    'features': features, 
}, period='P1D', auto_play = False).add_to(m)

m